In [111]:
import numpy as np
import pandas as pd
from functions import *
from read_from_file_df import *
from itertools import product

In [112]:
def sort_df(length, df):
    df = df.drop_duplicates().reset_index(drop=True)

    new_columns = [f'Col_{i}' for i in range(1, length + 1)]
    df_sort = df.sort_values(by = new_columns).reset_index(drop=True)
    for i in df_sort.columns:
        df_sort[i] = df_sort[i].astype(int)
        
    return df_sort

In [113]:

def build_data_to_df(size,amount_boards,amount_moves,num_dict,ignore_range,path,gen,reverse):
    os.makedirs(f'{path}\\{size}-{amount_boards}',exist_ok=True)
    df2 = split_board_to_series_df(size,amount_boards,amount_moves,num_dict,gen,ignore_range,reverse)
    df2_sort = sort_df(gen*size*size, df2)
    if reverse==True:
        name_df = f'{path}\\{size}-{amount_boards}\\{size}size_{amount_boards}boards_{gen}gen_reverse'
    else:
        name_df = f'{path}\\{size}-{amount_boards}\\{size}size_{amount_boards}boards_{gen}gen_non-reverse'
    df2_sort.to_pickle(f'{name_df}.pkl')
    print("\nfinish build df ",gen, reverse)

In [114]:
def split_board_to_series_df(size, amount_boards, amount_moves, num_dict, amount_board_in_series, ignore_range, reverse = False):
    new_columns = [f'Col_{i}' for i in range(1, amount_board_in_series*size*size + 1)]
    res_df = pd.DataFrame(columns=new_columns)

    for i in range(amount_boards):
        print_big_numbers(i)
        # path to read
        path_file = path(size, i, amount_moves, num_dict, amount_boards)
        # read the file
        df = read_file_to_df(path_file, size)
        #after we delete the repeat boards, we split the board to series
        if(len(df)>ignore_range):
            df = df[ignore_range:].drop_duplicates().reset_index(drop=True)
            new_row = df
            for _ in range(amount_board_in_series-1):
                new_col = df.iloc[1:].reset_index(drop=True)
                df = df.iloc[1:].reset_index(drop=True)
                if reverse==False:
                    new_row = pd.concat([new_row.iloc[:-1],new_col],axis=1)
                else:
                    new_row = pd.concat([new_col,new_row.iloc[:-1]],axis=1)
            new_row.columns = new_columns
            res_df = pd.concat([res_df,new_row])
    return res_df

In [115]:
def path(size, number, amount_moves, num_dict, amount_boards):
    return f"boards_{size}_{amount_boards}_{amount_moves}_{num_dict}\\{str(number % num_dict)}\\{str(size)}-{str(number)}-{str(amount_moves)}boards.bnr"

In [116]:
def read_file_to_df(pathFile, size):
    # Define the file path
    #path_file = 'C:\\GameOfLife\\boards\\' + path(SIZE, READ_FILE, AMOUNT_MOVES, NUM_DICT)
    path_file = PATH_BOARDS + pathFile

    # Read binary data into a numpy array of 8-bit integers
    binary_data = np.fromfile(path_file, dtype=np.uint8)

    # Convert each integer to its binary representation
    binary_data_binary_repr = np.array([list(np.binary_repr(x, width=8)) for x in binary_data], dtype=np.uint8).flatten()

    # Calculate the total number of elements in each game board
    board_elements = size**2

    # Calculate the number of boards in the file
    num_boards = len(binary_data_binary_repr) // board_elements

    # Reshape the binary data to create a 3D array with dimensions (num_boards, board_size[0], board_size[1])
    boards_data = binary_data_binary_repr[:num_boards * board_elements].reshape((num_boards, size, size))

    # Create a list of column names based on the board size
    column_names = [f'({i},{j})' for i in range(size) for j in range(size)]

    return pd.DataFrame(boards_data.reshape((num_boards, board_elements)), columns=column_names)

In [117]:
SIZE = 4
AMOUNT_BOARDS = 1000
AMOUNT_MOVES = 100
NUM_DICT = 1
for gen,reverse in product(range(2,5),[True,False]):
    print(gen,reverse)
    build_data_to_df(SIZE, AMOUNT_BOARDS, AMOUNT_MOVES, NUM_DICT, IGNORE_RANGE, PATH_DF, gen, reverse)

2 True
0 
50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 
finish build df  2 True
2 False
0 
50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 
finish build df  2 False
3 True
0 
50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 
finish build df  3 True
3 False
0 
50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 
finish build df  3 False
4 True
0 
50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 
finish build df  4 True
4 False
0 
50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 
finish build df  4 False
